<a href="https://colab.research.google.com/github/tarupathak30/machine_learning_algorithms/blob/main/loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [1]:
from google.colab import files
uploaded = files.upload()

Saving loan_data.csv to loan_data.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/loan_data.csv')

In [4]:
df.shape

(45000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [6]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [8]:
df.isnull().sum()

,0
person_age,0
person_gender,0
person_education,0
person_income,0
person_emp_exp,0
person_home_ownership,0
loan_amnt,0
loan_intent,0
loan_int_rate,0
loan_percent_income,0


1. person_gender is non-ordinal feature, so using label encoder for it. Binary Mapping (0/1) is practical.

2. person_education is ordinal data, to preserve education hierarchy, needed to use Label Encoding.

3. person_home_ownership is non-ordinal data, so i am going to apply one hot encoder.

4. loan_intent, is non-ordinal to my perception, hence one hot encoder.

5. previous_loan_default_on_file is not ordinal but falls under binary category, can do 0/1 by using label encoder can save dimensions after one hot encoder.

In [9]:
x = df.drop(columns=['loan_status'], axis=1)
y = df['loan_status']

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Let's up first discuss "Why One-Hot Encoding for linear models?"

Intuition 1 : Because linear models(like Logistic Regression or SVM with linear kernel) assume linear relationships between features. If we use Label Encoding for a non-ordinal categorical variable, we are seemed to be lying to the model by giving it fake order.

Intuition 2 : Like me, you must be thinking how? Where did we lie? Imagine feeding the label encoded data to the linear model to logistic regression. It will start creating real numerical distance or order between those categories, haven't we learned they involve mathematical equations involving numerical magnitudes and directions.


Let's see an example, we labeled own=0, rent=1, mortgage=2, what do you think it will do? own<rent<mortgage! Does it seem alright to you in any case in here with linear models?

But, in which case it's alright for tree-based model?

Intuition 3 : Now show up with why tree-based models? Because they make splits based on feature values, not magnitude or direction.

They never assume distance or linearity, just splits!

For example : own = 2, rent = 1, home = 0,

That means, if ownership < 2, do this:
else do that!

Did you find even a speck of linearity in here?

Hey, a quick reminder we will not use Naive Bayes' here (even though it is a classification algorithm) because the features are not categorically heavy. Here we have only 5 categorical features with 1 target column out of 14 features.

# Choosing Best Algorithm One by One

Now, we spent alot of time on encoding, what if I start with Choosing the best algorithm for our data.

Hey, a quick reminder we will not use Naive Bayes' here (even though it is a classification algorithm) because the features are not categorically heavy. Here we have only 5 categorical features with 1 target column out of 14 features.



Which one do you think we should not take? My bet is on KNN.


But why? It performs distance calculation for every prediction, which becomes very slow with thousands of rows(ours has 45000). And yes it doesn't scale well in high dimensional spaces(we are supposed to have them after encoding).

I think non-linear SVM will make the operation too slow, we will limit us to linear SVC(Support Vector Classification)

# Applying the Algorithms through pipeline

Let's separate the categorical and numerical columns

In [17]:
categorical = []
numerical = []
for col in x.columns:
  if x[col].dtype == 'object':
    categorical.append(col)
  else:
    numerical.append(col)

In [18]:
categorical

['person_gender',
 'person_education',
 'person_home_ownership',
 'loan_intent',
 'previous_loan_defaults_on_file']

In [19]:
numerical

['person_age',
 'person_income',
 'person_emp_exp',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length',
 'credit_score']

mandatory preprocessing setup

In [22]:
label_cols = ['person_gender', 'person_education', 'previous_loan_defaults_on_file']
ohe_cols = ['person_home_ownership', 'loan_intent']

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

In [28]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical),
    ('categorical_label', OrdinalEncoder(), label_cols),
    ('categorical_onehot', OneHotEncoder(), ohe_cols)
], remainder='passthrough')

building pipelines for each model

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

In [34]:
# logistic regression pipeline
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier_lor', LogisticRegression())
])

In [35]:
# Decision Tree
pipeline_dtree = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier_dtree', DecisionTreeClassifier())
])

In [36]:
# Linear Ṣupport Vector Classifier
pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier_svc', LinearSVC())
])

In [39]:
pipeline_lr.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score']),
                                                 ('categorical_label',
                                                  OrdinalEncoder(),
                                                  ['person_gender',
                                                   'person_education',
                                                   'previous_loan_defaults_on_file']),
                                                 ('categorical_onehot',
                                                  OneHotEncoder(),
                                                  ['person_home_ownership',
                                                   'loan_intent'])])),
                ('classifier_lor', LogisticRegression())])

In [40]:
pipeline_dtree.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score']),
                                                 ('categorical_label',
                                                  OrdinalEncoder(),
                                                  ['person_gender',
                                                   'person_education',
                                                   'previous_loan_defaults_on_file']),
                                                 ('categorical_onehot',
                                                  OneHotEncoder(),
                                                  ['person_home_ownership',
                                                   'loan_intent'])])),
                ('classifier_dtree', DecisionTreeClassifier())])

In [41]:
pipeline_svc.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score']),
                                                 ('categorical_label',
                                                  OrdinalEncoder(),
                                                  ['person_gender',
                                                   'person_education',
                                                   'previous_loan_defaults_on_file']),
                                                 ('categorical_onehot',
                                                  OneHotEncoder(),
                                                  ['person_home_ownership',
                                                   'loan_intent'])])),
                ('classifier_svc', LinearSVC())])

In [42]:
y_pred_lr = pipeline_lr.predict(x_test)
y_pred_dtree = pipeline_dtree.predict(x_test)
y_pred_svc = pipeline_svc.predict(x_test)

Now, It's time to compare the scores

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
print("Accuracy Score Logistic Regression ", accuracy_score(y_test, y_pred_lr))
print("Accuracy Score Decision Tree ", accuracy_score(y_test, y_pred_dtree))
print("Accuracy Score Support Vector Classifier ", accuracy_score(y_test, y_pred_svc))

Accuracy Score Logistic Regression  0.8938888888888888
Accuracy Score Decision Tree  0.9025555555555556
Accuracy Score Support Vector Classifier  0.8935555555555555


Let's cross validate now, to check the robustness

In [45]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline_lr, x, y, cv=5)
print("Cross Validation Accuracy Logistic Regression: ", scores.mean())

Cross Validation Accuracy :  0.8954000000000001


In [46]:
scores = cross_val_score(pipeline_svc, x, y, cv=5)
print("Cross Validation Accuracy Support Vector Classifier: ", scores.mean())

Cross Validation Accuracy Support Vector Classifier:  0.8954222222222222


In [47]:
scores = cross_val_score(pipeline_dtree, x, y, cv=5)
print("Cross Validation Accuracy : ", scores.mean())

Cross Validation Accuracy :  0.8958444444444444


We could not the change only in the accuracy of Decision Tree, rest of them are nearly same, right?

What's next? Confusion Matrix + Classification Report

In [48]:
from sklearn.metrics import classification_report, confusion_matrix

In [49]:
print("Confusion Matrix for Logistic Regression")
print(confusion_matrix(y_test, y_pred_lr))
print("Classification Report")
print(classification_report(y_test, y_pred_lr))


Confusion Matrix for Logistic Regression
[[6554  436]
 [ 519 1491]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      6990
           1       0.77      0.74      0.76      2010

    accuracy                           0.89      9000
   macro avg       0.85      0.84      0.84      9000
weighted avg       0.89      0.89      0.89      9000



In [51]:
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test, y_pred_dtree))
print("Classification Report")
print(classification_report(y_test, y_pred_dtree))

Confusion Matrix for Decision Tree
[[6523  467]
 [ 410 1600]]
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      6990
           1       0.77      0.80      0.78      2010

    accuracy                           0.90      9000
   macro avg       0.86      0.86      0.86      9000
weighted avg       0.90      0.90      0.90      9000



In [52]:
print("Confusion Matrix for Support Vector Classifier")
print(confusion_matrix(y_test, y_pred_svc))
print("Classification Report")
print(classification_report(y_test, y_pred_svc))

Confusion Matrix for Support Vector Classifier
[[6553  437]
 [ 521 1489]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      6990
           1       0.77      0.74      0.76      2010

    accuracy                           0.89      9000
   macro avg       0.85      0.84      0.84      9000
weighted avg       0.89      0.89      0.89      9000



Let's do some Hyperparameter Tuning. It's beneficial for accuracy for each model.

In [53]:
from sklearn.model_selection import GridSearchCV

In [70]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats



In [71]:
param_grid = {
    'classifier_lor__penalty' : ['l1', 'l2', 'elasticnet', None],
    'classifier_lor__solver' : ['lbfgs', 'liblinear', 'sag', 'saga'],
    'classifier_lor__max_iter' : [50, 100, 200, 1000, 2000, 5000, 10000]
}

In [72]:
random_search_lr = RandomizedSearchCV(pipeline_lr, param_distributions=param_grid,
                                   n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search_lr.fit(x_train, y_train)

y_pred_lr1 = random_search_lr.predict(x_test)

print("Accuracy Score : ", accuracy_score(y_test, y_pred_lr1))

Accuracy Score :  0.8936666666666667


Not Actually beneficial maybe :) Next for Support Vector Classifier

In [73]:
param_grid = {
    'classifier_svc__penalty' : ['l1', 'l2'], #penalty norm
    'classifier_svc__C' : [0.01, 0.1, 1, 10, 100], #regularization parameter
     'classifier_svc__max_iter' : [1000, 2000, 5000]
}

In [74]:
random_search_svc = RandomizedSearchCV(pipeline_svc, param_distributions=param_grid,
                                   n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search_svc.fit(x_train, y_train)

y_pred_svc1 = random_search_svc.predict(x_test)

print("Accuracy Score : ", accuracy_score(y_test, y_pred_svc1))

Accuracy Score :  0.8937777777777778


It disappointed here because of the random nature of RandomizedSearchCV, possibly the case of overfitting to CV folds and hence failing on the test data.  